# Creating Your First Agent Society

You can also check this cookbook in colab [here](https://colab.research.google.com/drive/1cmWPxXEsyMbmjPhD2bWfHuhd_Uz6FaJQ?usp=sharing)

## Philosophical Bits



> *What magical trick makes us intelligent? The trick is that there is no trick. The power of intelligence stems from our vast diversity, not from any single, perfect principle.*
>
> -- Marvin Minsky, The Society of Mind, p. 308

In this section, we will take a spite of the task-oriented `RolyPlaying()` class. We design this in an instruction-following manner. The essence is that to solve a complex task, you can enable two communicative agents collabratively working together step by step to reach solutions. The main concepts include:

- **Task**: a task can be as simple as an idea, initialized by an inception prompt.

- **AI User**: the agent who is expected to provide instructions.

- **AI Assistant**: the agent who is expected to respond with solutions that fulfills the instructions.

## Quick Start
Let's first play with a `ChatAgent` instance by simply initialize it with a system message and interact with user messages.

### 🕹 Step 0: Prepartions

In [ ]:
%pip install "camel-ai==0.2.1"

In [9]:
# Import necessary classes
from camel.societies import RolePlaying
from camel.types import TaskType, ModelType, ModelPlatformType
from camel.configs import ChatGPTConfig
from camel.models import ModelFactory
import os

model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI,
    model_type=ModelType.GPT_4O_MINI,
    model_config_dict=ChatGPTConfig(temperature=0.0).as_dict(),
)

### Setting Up API Keys

You'll need to set up your API keys for OpenAI.

In [4]:
import os
from getpass import getpass

# Prompt for the API key securely
openai_api_key = getpass('Enter your API key: ')
os.environ["OPENAI_API_KEY"] = openai_api_key

Enter your API key: ··········


### 🕹 Step 1: Configure the Role-Playing Session


#### Set the `Task` Arguments

In [10]:
task_kwargs = {
    'task_prompt': 'Develop a plan to TRAVEL TO THE PAST and make changes.',
    'with_task_specify': True,
    'task_specify_agent_kwargs': {'model': model}
}

#### Set the `User` Arguments
You may think the user as the `instruction sender`.

In [11]:
user_role_kwargs = {
    'user_role_name': 'an ambitious aspiring TIME TRAVELER',
    'user_agent_kwargs': {'model': model}
}

#### Set the `Assistant` Arguments
Again, you may think the assistant as the `instruction executor`.

In [12]:
assistant_role_kwargs = {
    'assistant_role_name': 'the best-ever experimental physicist',
    'assistant_agent_kwargs': {'model': model}
}

### Step 2: Kickstart Your Society
Putting them altogether – your role-playing session is ready to go!

In [13]:
society = RolePlaying(
    **task_kwargs,             # The task arguments
    **user_role_kwargs,        # The instruction sender's arguments
    **assistant_role_kwargs,   # The instruction receiver's arguments
)

### Step 3: Solving Tasks with Your Society
Hold your bytes. Prior to our travel, let's define a small helper function.

In [14]:
def is_terminated(response):
    """
    Give alerts when the session shuold be terminated.
    """
    if response.terminated:
        role = response.msg.role_type.name
        reason = response.info['termination_reasons']
        print(f'AI {role} terminated due to {reason}')

    return response.terminated

Time to chart our course – writing a simple loop for our society to proceed:

In [15]:
def run(society, round_limit: int=10):

    # Get the initial message from the ai assistant to the ai user
    input_msg = society.init_chat()

    # Starting the interactive session
    for _ in range(round_limit):

        # Get the both responses for this round
        assistant_response, user_response = society.step(input_msg)

        # Check the termination condition
        if is_terminated(assistant_response) or is_terminated(user_response):
            break

        # Get the results
        print(f'[AI User] {user_response.msg.content}.\n')
        print(f'[AI Assistant] {assistant_response.msg.content}.\n')

        # Check if the task is end
        if 'CAMEL_TASK_DONE' in user_response.msg.content:
            break

        # Get the input message for the next round
        input_msg = assistant_response.msg

    return None

In [16]:
run(society)

[AI User] Instruction: Research the principles of quantum entanglement and wormhole theory to understand their potential applications in time travel.  
Input: None.

[AI Assistant] Solution: To understand the potential applications of quantum entanglement and wormhole theory in time travel, we need to delve into the fundamental principles of each concept.

1. **Quantum Entanglement**:
   - **Definition**: Quantum entanglement is a phenomenon where two or more particles become interconnected in such a way that the state of one particle instantly influences the state of another, regardless of the distance separating them.
   - **Key Principles**:
     - **Non-locality**: Changes to one entangled particle affect its partner instantaneously, suggesting a form of communication that transcends space.
     - **Superposition**: Particles exist in multiple states until measured, which could theoretically allow for multiple timelines or outcomes.
   - **Potential Applications in Time Travel**:
 